In [ ]:
#pip3 install -U transformers


In [ ]:
#pip install --upgrade transformers

In [1]:
## Importing necessary Libraries
import pandas as pd
import os
import json
import glob

In [ ]:
# file_path = 'RQ1/preprocessed_dataset_for_nf_core_pipelines.json'

# with open(file_path, "r") as f:
#     data = json.load(f)
    
# preprocessed_df = pd.json_normalize(data)
# preprocessed_df

In [2]:
preprocessed_df = pd.read_json('../Projects/preprocessed_fl_dataset.json')
preprocessed_df

,id,title,text,created_at,closed_at,state
0,2.775963e+09,little mistake in the repo example change logs...,little mistake repo example change logsroot lo...,2025-01-08 17:42:25,NaT,open
1,1.985601e+09,How to use makeplot function in utilsplots.py?,use makeplot function utilsplotspy hi thank wo...,2023-11-09 13:14:38,2023-11-13 07:39:02,closed
2,1.395683e+09,Any plan to provide a license?,plan provide license hello thank share helpful...,2022-10-04 05:22:04,2022-10-06 14:32:55,closed
3,1.307041e+09,Question about MOCHA,question mocha hi omarfoq openreviewhttpsopenr...,2022-07-17 09:19:41,2022-07-18 13:46:23,closed
4,1.258391e+09,Args for synthetic datasets,args synthetic dataset thank interesting work ...,2022-06-02 16:02:33,2022-06-03 02:53:55,closed
...,...,...,...,...,...,...
9111,1.814080e+09,,33analysis ALA 456,2023-07-20 14:06:08,2023-09-02 10:51:32,closed
9112,1.706164e+09,,ps cid int loss nnmodule traindata ListTuple b...,2023-05-11 16:29:46,2023-05-28 02:36:26,closed
9113,1.670350e+09,Could you please provide the BibTeX?,could please provide BibTeX would like cite wo...,2023-04-17 03:44:45,2023-05-21 06:15:13,closed
9114,1.648384e+09,Reproducing Table 2.,reproduce table 2 hello first thank provide wo...,2023-03-30 22:01:03,2023-05-07 11:00:26,closed


Pre-calculate Embeddings

In [3]:
from sentence_transformers import SentenceTransformer
## Precalculate Embedding
embedding_model = SentenceTransformer("all-mpnet-base-v2")

/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sahand/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
preprocessed_df['text'] = preprocessed_df['text'].astype(str)

In [5]:
# Try encoding
try:
    embeddings_text = embedding_model.encode(preprocessed_df['text'].tolist(), show_progress_bar=True)
except KeyError as e:
    print(f"KeyError encountered: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Batches: 100%|██████████| 285/285 [01:03<00:00,  4.49it/s]


In [ ]:
#pip install --upgrade numba

In [6]:
from bertopic import BERTopic

In [7]:
from umap import UMAP
from hdbscan import HDBSCAN
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import itertools

In [8]:
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [9]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [10]:
docs = preprocessed_df['text'].tolist()
docs

['little mistake repo example change logsroot logsdir argument name logsroot change logsdir ok example   python3 FedEMrunexperimentpy cifar10 FedAvg nlearner 1 nround 200 bz 128 lr 001 lrscheduler multistep logfreq 5 device cuda optimizer sgd seed 1234 logsdir content verbose 1',
 'use makeplot function utilsplotspy hi thank work could release script use makeplot function utilsplotspy look forward it',
 'plan provide license hello thank share helpful work plan declare type license code hopefully would like publish experimental code contain custmize part code follow manner specify license',
 'question mocha hi omarfoq openreviewhttpsopenreviewnetforumidycqx6zhezrp say implement MOCHA algorithm python experiment it updata part code repository hace implement MOCHA algorithm recently confuse SDCA update part official matlab source code would like see implementation thank much',
 'args synthetic dataset thank interesting work really appreciate try reuse dataset provide want generate synthet

In [11]:
results_2=[]

for n_neighbors in [10,15,20,25, 30,35,40,45, 50]:
    for n_components in [5, 10, 15,20,25,30]:
        for min_cluster_size in [10,15,30,50,70,90, 110, 140,170]:
            try:
                umap_model = UMAP(
                    n_neighbors=n_neighbors,
                    n_components=n_components,
                    metric="cosine",
                    random_state=42,
                )

                hdbscan_model = hdbscan.HDBSCAN(
                    min_cluster_size=min_cluster_size,
                    metric="euclidean",
                    cluster_selection_method="eom",
                )

                topic_model = BERTopic(
                    umap_model=umap_model,
                    hdbscan_model=hdbscan_model,
                    calculate_probabilities=False,
                    verbose=False,
                )

                topics, _ = topic_model.fit_transform(docs)

                # Collect words for coherence
                top_words = [
                    [word for word, _ in topic_model.get_topic(topic)]
                    for topic in topic_model.get_topic_freq().Topic
                ]

                tokenized_docs = [doc.split() for doc in docs]
                dictionary = Dictionary(tokenized_docs)
                coherence_model = CoherenceModel(
                    topics=top_words,
                    texts=tokenized_docs,
                    dictionary=dictionary,
                    coherence="c_v",
                )
                coherence = coherence_model.get_coherence()

                results_2.append(
                    {
                        "n_neighbors": n_neighbors,
                        "n_components": n_components,
                        "min_cluster_size": min_cluster_size,
                        "coherence": coherence,
                    }
                )
                print(
                    f"✅ n_neighbors={n_neighbors}, n_components={n_components}, "
                    f"min_cluster_size={min_cluster_size}, coherence={coherence:.4f}"
                )

            except Exception as e:
                print(
                    f"⚠️ Skipped: n_neighbors={n_neighbors}, "
                    f"n_components={n_components}, min_cluster_size={min_cluster_size} "
                    f"({type(e).__name__}: {e})"
                )
                continue


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:3

✅ n_neighbors=10, n_components=5, min_cluster_size=10, coherence=0.3640


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:3

✅ n_neighbors=10, n_components=5, min_cluster_size=15, coherence=0.3986


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:3

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:3

✅ n_neighbors=20, n_components=30, min_cluster_size=15, coherence=0.2788


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/sahand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:3

✅ n_neighbors=20, n_components=30, min_cluster_size=30, coherence=0.2788


KeyboardInterrupt: 

In [ ]:
df_results_2 = pd.DataFrame(results)
df_results_2.to_csv('../hyper_parameters_value.csv')
print(df_results_2.sort_values(by="coherence", ascending=False))